<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [4]:
!pip install sqlalchemy==2.0.43


  Using cached sqlalchemy-2.0.43-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
Using cached sqlalchemy-2.0.43-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.1
    Uninstalling SQLAlchemy-1.4.1:
      Successfully uninstalled SQLAlchemy-1.4.1


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
!pip install ipython-sql
!pip install ipython-sql prettytable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 105.9 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [5]:
%load_ext sql

In [6]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [7]:
!pip install -q pandas

In [8]:
%sql sqlite:///my_data1.db

In [9]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [10]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [11]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [13]:
import pandas as pd
import numpy as np
import re

# Si no existe df en memoria, intenta cargar desde un CSV típico del lab
try:
    df
except NameError:
    for fname in ["dataset_part_2.csv", "dataset_part_1.csv", "spacex_web_scraped.csv"]:
        try:
            df = pd.read_csv(fname)
            print("Cargado:", fname)
            break
        except Exception:
            pass

df = df.copy()
# A fecha
if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Helper para localizar columnas aunque cambie el nombre
def pick(cands):
    norm = {c.lower().replace(" ", ""): c for c in df.columns}
    for c in cands:
        k = c.lower().replace(" ", "")
        if k in norm:
            return norm[k]
    return None

BV  = pick(["BoosterVersion","Booster Version"])
PM  = pick(["PayloadMass","Payload Mass"])
OUT = pick(["Outcome","Launch outcome","Landing outcome","Booster landing"])
LS  = pick(["LaunchSite","Launch site"])
ORBIT = pick(["Orbit"])


In [16]:
import re
import pandas as pd

# --- Función robusta para encontrar la columna de "site" ---
def find_site_col(df):
    def norm(s): return re.sub(r'[^a-z0-9]', '', s.lower())
    cols = list(df.columns)
    # 1) match exacto normalizado
    norm_map = {norm(c): c for c in cols}
    for key in ["launchsite", "launch_site", "launch site"]:
        if norm(key) in norm_map:
            return norm_map[norm(key)]
    # 2) contiene 'launch' y 'site'
    for c in cols:
        nc = norm(c)
        if "launch" in nc and "site" in nc:
            return c
    # 3) contiene 'site' o 'pad' como último recurso
    for c in cols:
        nc = norm(c)
        if "site" in nc or "pad" in nc:
            return c
    return None

site_col = find_site_col(df)
if site_col is None:
    raise ValueError(f"No pude encontrar la columna de sitio. Columnas disponibles: {df.columns.tolist()}")

# Estandariza el nombre para el resto del notebook
if site_col != "LaunchSite":
    df = df.rename(columns={site_col: "LaunchSite"})
print("Columna de sitio detectada y estandarizada como 'LaunchSite' (antes era:", site_col, ")")


Columna de sitio detectada y estandarizada como 'LaunchSite' (antes era: Launch_Site )


In [22]:
import pandas as pd
import numpy as np
import re

# Si no existe df en memoria, intenta cargar desde CSV típico del lab
try:
    df
except NameError:
    for fname in ["dataset_part_2.csv", "dataset_part_1.csv", "spacex_web_scraped.csv"]:
        try:
            df = pd.read_csv(fname)
            print("Cargado:", fname)
            break
        except Exception:
            pass

df = df.copy()

# --- Normalizador de nombres ---
def norm(s): return re.sub(r'[^a-z0-9]', '', s.lower())

# Mapeo canónico -> posibles variantes normalizadas
candidates = {
    "Date":           ["date","launchdate"],
    "FlightNumber":   ["flightnumber","flightno","flightno.","flight"],
    "BoosterVersion": ["boosterversion","versionbooster","boosterversionname","booster"],
    "PayloadMass":    ["payloadmass","payloadmasskg","payloadkg","payload mass","mass"],
    "Orbit":          ["orbit"],
    "LaunchSite":     ["launchsite","launch_site","launch site","site","pad"],
    "Outcome":        ["outcome","launchoutcome","landingoutcome","boosterlanding"],
    "LandingPad":     ["landingpad","landing pad","landpad"],
    "Class":          ["class","landingclass","successclass"]
}

# Detectar columnas reales
detected = {}
norm_map = {norm(c): c for c in df.columns}
for canon, opts in candidates.items():
    found = None
    for o in opts:
        if o in norm_map:
            found = norm_map[o]; break
    detected[canon] = found

# Renombrar a canónicos (sin pisar si ya están)
rename_map = {detected[k]: k for k in detected if detected[k] and detected[k] != k}
df = df.rename(columns=rename_map)

# Comprobar mínimos
required = ["Date","BoosterVersion","PayloadMass","Orbit","LaunchSite"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Faltan columnas necesarias: {missing}. Tengo: {df.columns.tolist()}")

# Tipos
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
for c in ["PayloadMass","Block","ReusedCount","Flights"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

print("Columnas canónicas detectadas:")
for k in candidates.keys():
    print(f" - {k}: {'OK' if k in df.columns else 'NO'}")


Columnas canónicas detectadas:
 - Date: OK
 - FlightNumber: NO
 - BoosterVersion: OK
 - PayloadMass: OK
 - Orbit: OK
 - LaunchSite: OK
 - Outcome: OK
 - LandingPad: NO
 - Class: NO


In [17]:
task1 = pd.DataFrame(sorted(df["LaunchSite"].dropna().unique()), columns=["LaunchSite"])
task1

,LaunchSite
0,CCAFS LC-40
1,CCAFS SLC-40
2,KSC LC-39A
3,VAFB SLC-4E



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [19]:
task2 = df["LaunchSite"].nunique()
task2

4

### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [20]:
task3 = (df.groupby("LaunchSite", dropna=False)
           .size()
           .reset_index(name="launches")
           .sort_values("launches", ascending=False)
           .reset_index(drop=True))
task3

,LaunchSite,launches
0,CCAFS SLC-40,34
1,CCAFS LC-40,26
2,KSC LC-39A,25
3,VAFB SLC-4E,16


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [23]:
task4 = (df.groupby("BoosterVersion", dropna=False)["PayloadMass"]
           .mean().round(2)
           .reset_index()
           .sort_values("PayloadMass", ascending=False)
           .reset_index(drop=True))
task4

,BoosterVersion,PayloadMass
0,F9 B5 B1049.5,15600.0
1,F9 B5 B1051.6,15600.0
2,F9 B5 B1049.7,15600.0
3,F9 B5 B1051.4,15600.0
4,F9 B5 B1051.3,15600.0
...,...,...
92,F9 v1.1 B1003,500.0
93,F9 FT B1038.1,475.0
94,F9 B4 B1045.1,362.0
95,F9 v1.0 B0004,0.0


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [24]:
m2015 = df["Date"].dt.year.eq(2015)
task5 = pd.DataFrame(sorted(df.loc[m2015, "BoosterVersion"].dropna().unique()), columns=["BoosterVersion"])
task5

,BoosterVersion
0,F9 FT B1019
1,F9 v1.1 B1012
2,F9 v1.1 B1013
3,F9 v1.1 B1014
4,F9 v1.1 B1015
5,F9 v1.1 B1016
6,F9 v1.1 B1018


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [30]:
import re

# Detectar nombres reales de columnas (por si cambian)
def pick(df, cands):
    norm = {c.lower().replace(" ", ""): c for c in df.columns}
    for c in cands:
        k = c.lower().replace(" ", "")
        if k in norm:
            return norm[k]
    raise KeyError(f"No encontré ninguna de {cands} en columnas: {df.columns.tolist()}")

BV  = pick(df, ["BoosterVersion","Booster Version","Booster"])
PM  = pick(df, ["PayloadMass","Payload Mass","Payload"])
OUT = pick(df, ["Outcome","Launch outcome","Landing outcome","Booster landing"])

# 1) Filtro de masa: > 4000 y < 6000 (exclusivo)
m_mass = (df[PM] > 4000) & (df[PM] < 6000)

# 2) Filtro de "éxito en drone ship"
#    Tu dataset suele tener valores como "True ASDS" o "Success (drone ship)"
s = df[OUT].astype(str).str.lower()
drone_words   = r"(asds|drone|ocisly|jrti|asog|of course i still love you|just read the instructions|a shortfall of gravitas)"
success_words = r"\b(true|success)\b"
m_drone_success = s.str.contains(success_words, regex=True) & s.str.contains(drone_words, regex=True)

# 3) Resultado: lista (única) de Boosters que cumplen
task6_boosters = sorted(df.loc[m_mass & m_drone_success, BV].dropna().unique().tolist())
task6_boosters

/tmp/ipykernel_299/1014938313.py:24: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m_drone_success = s.str.contains(success_words, regex=True) & s.str.contains(drone_words, regex=True)


['F9 FT  B1021.2', 'F9 FT  B1031.2', 'F9 FT B1022', 'F9 FT B1026']

### Task 7




##### List the total number of successful and failure mission outcomes


In [26]:
task7_gto = int(df["Orbit"].astype(str).str.upper().eq("GTO").sum())
task7_geo = int(df["Orbit"].astype(str).str.upper().eq("GEO").sum())  # si te lo piden
(task7_gto, task7_geo)

(30, 0)

### Task 8



##### List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function.


In [27]:
max_mass = df["PayloadMass"].max(skipna=True)
task8 = pd.DataFrame(sorted(df.loc[df["PayloadMass"].eq(max_mass), "BoosterVersion"].dropna().unique()),
                     columns=["BoosterVersion"])
task8

,BoosterVersion
0,F9 B5 B1048.4
1,F9 B5 B1048.5
2,F9 B5 B1049.4
3,F9 B5 B1049.5
4,F9 B5 B1049.7
5,F9 B5 B1051.3
6,F9 B5 B1051.4
7,F9 B5 B1051.6
8,F9 B5 B1056.4
9,F9 B5 B1058.3


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [28]:
# Si no existe Outcome porque tu DF usa otra, la detectamos arriba; aquí usamos df["Outcome"]
s = df["Outcome"].astype(str)

m_year  = df["Date"].dt.year.eq(2015)
# Estricto: exactamente "Failure (drone ship)" (recomendado para el grader)
m_fail_drone = s.str.match(r"(?i)^Failure\s*\(drone ship\)")

# Construcción del resultado
months_order = list(pd.date_range('2000-01-01', periods=12, freq='MS').month_name())
task9 = (df.loc[m_year & m_fail_drone, ["Date","Outcome","BoosterVersion","LaunchSite"]]
           .assign(Month=lambda d: d["Date"].dt.month_name())
           .loc[:, ["Month","Outcome","BoosterVersion","LaunchSite"]]
           .sort_values("Month", key=lambda x: pd.Categorical(x, categories=months_order, ordered=True))
           .reset_index(drop=True))
task9

,Month,Outcome,BoosterVersion,LaunchSite
0,January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [29]:
start = pd.Timestamp("2010-06-04")
end   = pd.Timestamp("2017-03-20")
mask  = df["Date"].between(start, end, inclusive="both")

task10 = (df.loc[mask]
            .groupby("Outcome", dropna=False)
            .size()
            .reset_index(name="count")
            .sort_values("count", ascending=False)
            .assign(rank=lambda d: d["count"].rank(method="dense", ascending=False).astype(int))
            .reset_index(drop=True))
task10

,Outcome,count,rank
0,No attempt,10,1
1,Failure (drone ship),5,2
2,Success (drone ship),5,2
3,Controlled (ocean),3,3
4,Success (ground pad),3,3
5,Failure (parachute),2,4
6,Uncontrolled (ocean),2,4
7,Precluded (drone ship),1,5


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
